In [89]:
pip install pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [90]:
  !pip install -qU \
  transformers==4.31.0 \
  sentence-transformers==2.2.2 \
  datasets==2.14.0 \
  accelerate==0.21.0 \
  einops==0.6.1 \
  langchain==0.0.240 \
  xformers==0.0.20 \
  bitsandbytes==0.41.0\
  InstructorEmbedding

In [91]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [92]:
!pip install --user mysql-connector-python

In [93]:
pip install ctransformers

Note: you may need to restart the kernel to use updated packages.


In [94]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.chat_models import ChatOpenAI
from transformers import AutoTokenizer
from dotenv import load_dotenv
import os
import timeit
import sys
import mysql.connector

In [95]:
load_dotenv()
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY','1acea4e0-c639-4577-b431-6d607796e5d5')
PINECONE_API_ENV=os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [96]:
mysql_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="N090603",
    database="railway"  # Change this to your database name
)

In [97]:
mysql_cursor = mysql_connection.cursor()

In [98]:
mysql_query = "SELECT * FROM rschedule" 

In [99]:
mysql_cursor.execute(mysql_query)

In [100]:
rows = mysql_cursor.fetchall()

In [101]:
mysql_cursor.close()
mysql_connection.close()

In [102]:
file_path = 'railway_schedule.txt'

# Open the file for writing (create it if it doesn't exist)
with open(file_path, 'w') as file:
    # Iterate through the rows of data and write each row as text
    for row in rows:
        # Convert the row to a string (adjust the formatting as needed)
        row_text = f"Train No: {row[0]}, Train Name: {row[1]}, SEQ: {row[2]}, Station Code: {row[3]}, Station Name: {row[4]}, Arrival Time: {row[5]},Departure Time: {row[6]},Distance: {row[7]},Source Station: {row[8]},Source Station Name: {row[9]},Destination Station: {row[10]},Destination Station Name: {row[11]}"

        # Write the row to the file
        file.write(row_text)

# Close the file
file.close()

In [103]:
documents = []

# Open the file for reading
with open(file_path, 'r') as file:
    # Read each line (document) from the file
    for line in file:
        # Append the line (document) to the list of documents, removing leading/trailing whitespace
        documents.append(line.strip())

In [104]:
input_text = "\n".join(documents)
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
tokenized_text = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512, padding="max_length")
input_ids = tokenized_text["input_ids"]
input_text = tokenizer.decode(input_ids[0])

In [105]:
# Assuming 'documents' is a list of strings from the text file

# Initialize a list to store the split documents
split_docs = []

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

# Split each line/document into smaller chunks
for document in documents:
    split_chunks = text_splitter.split_text(input_text)
    split_docs.extend(split_chunks)

# Now, 'split_docs' contains the smaller text chunks


In [106]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [107]:
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="railway-chatbot"

In [108]:
docsearch = Pinecone.from_texts(split_docs, embeddings, index_name=index_name)

In [109]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that I don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [110]:
chain_type_kwargs={"prompt": PROMPT}

llm=CTransformers(model="llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [111]:
qa=RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(search_kwargs={'k': 2}),return_source_documents=True, chain_type_kwargs=chain_type_kwargs)

In [113]:
while True:
    user_input=input(f"Input Prompt:")
    if user_input=='exit':
        print('Exiting')
        sys.exit()
    if user_input=='':
        continue
    result=qa({"query": user_input})
    print("Response : ", result["result"])

Input Prompt:what is the destination station for train no 107?
Response :  The destination station for train no 107 is madgoan jn.


KeyboardInterrupt: Interrupted by user